In [1]:
%autosave 300
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch


In this notebook we will design a custom resnet architecture as specified in session 10 ot ERA course

In [3]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchview import draw_graph
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
from torcheval.metrics import MulticlassAccuracy, BinaryAccuracy
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import (
    OneCycleLR,
    StepLR,
    ExponentialLR,
)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import scikitplot as skplt
import seaborn as sns
import torch.nn.functional as F
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
from torch_lr_finder import LRFinder

In [ ]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [ ]:
def plot_loss_accuracy(
    train_loss,
    val_loss,
    train_acc,
    val_acc,
    labels,
    colors,
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
    legend_font=5,
    fig_size=(16, 10),
    sub_plot1=(1, 2, 1),
    sub_plot2=(1, 2, 2),
):

    plt.rcParams["figure.figsize"] = fig_size
    plt.figure

    plt.subplot(sub_plot1[0], sub_plot1[1], sub_plot1[2])

    for i in range(len(train_loss)):
        x_train = range(len(train_loss[i]))
        x_val = range(len(val_loss[i]))

        min_train_loss = np.array(train_loss[i]).min()

        min_val_loss = np.array(val_loss[i]).min()

        plt.plot(
            x_train,
            train_loss[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(min_train_loss, labels[i]),
        )
        plt.plot(
            x_val,
            val_loss[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(min_val_loss, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("loss")
    plt.legend(loc=loss_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Loss")

    plt.subplot(sub_plot2[0], sub_plot2[1], sub_plot2[2])

    for i in range(len(train_acc)):
        x_train = range(len(train_acc[i]))
        x_val = range(len(val_acc[i]))

        max_train_acc = np.array(train_acc[i]).max()

        max_val_acc = np.array(val_acc[i]).max()

        plt.plot(
            x_train,
            train_acc[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(max_train_acc, labels[i]),
        )
        plt.plot(
            x_val,
            val_acc[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(max_val_acc, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("accuracy")
    plt.legend(loc=acc_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Accuracy")

    plt.show()
    plt.close()

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(
        self,
        patience=3,
        verbose=True,
        delta=1e-5,
        trace_func=print,
        path="models",
        model_name="model.pt",
    ):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.model_name = model_name
        self.trace_func = trace_func

    def __call__(self, val_loss, model, epoch):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(
                f"EarlyStopping counter: {self.counter} out of {self.patience}"
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            self.trace_func(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        os.makedirs(self.path, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(self.path, self.model_name))
        self.val_loss_min = val_loss


def get_device():
    """Get device (if GPU is available, use GPU)"""
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = get_device()
print(device)
batch_size = 512
epochs = 25
learning_rate = 1e-3

In [ ]:
# Specify the image classes
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [ ]:
# transform for the dataset using Compose from albumentations
def data_augmentations():
    """Data Augmentations for the CIFAR10 dataset"""
    train_transforms = alb.Compose(
        [
            alb.Resize(
                height=36, width=36, always_apply=True, interpolation=cv2.INTER_NEAREST
            ),
            alb.RandomCrop(height=32, width=32, always_apply=True),
            alb.Flip(p=0.5),
            alb.CoarseDropout(
                max_holes=1,
                max_height=8,
                max_width=8,
                min_holes=1,
                min_height=8,
                min_width=8,
                fill_value=[0.4914, 0.4822, 0.4465],
                mask_fill_value=None,
            ),
            alb.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010)),
            ToTensorV2(),
        ]
    )

    test_transforms = alb.Compose(
        [
            alb.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010)),
            ToTensorV2(),
        ]
    )
    return train_transforms, test_transforms

In [ ]:
# downloading train and test data
train = datasets.CIFAR10("./data", train=True, download=True)
test = datasets.CIFAR10("./data", train=False, download=True)

In [ ]:
class CifarDS(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [ ]:
train_transforms, test_transforms = data_augmentations()
train_dateset = CifarDS(train.data, train.targets, train_transforms)
test_dateset = CifarDS(test.data, test.targets, test_transforms)

In [ ]:
def get_data_loader(
    train_dateset, test_dateset, train_transforms, test_transforms, dataloader_args
):
    """Data loader for the CIFAR10 dataset"""
    # Loading custom datasets
    train_dateset = CifarDS(train_dateset.data, train_dateset.targets, train_transforms)
    test_dateset = CifarDS(test_dateset.data, test_dateset.targets, test_transforms)
    # train dataloader
    train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)
    # test dataloader
    test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)
    return train_loader, test_loader

In [ ]:
for data,label in train_dateset:
    print(data.shape)
    print(label)
    print(data.mean())
    break

In [ ]:
# Dataloaders
# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = (
    dict(shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)
    if device.type == "cuda"
    else dict(shuffle=True, batch_size=batch_size)
)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)

In [ ]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

## Resent Model

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
    ):
        super(ResnetBlock, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

        self.resnet_block1 = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=(3, 3),
                stride=(2, 2),
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=out_channels,
                out_channels=out_channels,
                kernel_size=(3, 3),
                stride=(1, 1),
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        """forward pass of the resnet block"""
        interm_op = self.conv_layer1(x)
        residual_op = self.resnet_block1(x)
        # print(interm_op.shape)
        # print(residual_op.shape)
        op = residual_op + interm_op
        return op

In [ ]:
resnet_block = ResnetBlock(64,128).to(device)
summary(resnet_block, input_size=(64, 32, 32))

In [ ]:
class ResnetNetwork(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResnetNetwork, self).__init__()
        # PrepLayer - Conv 3x3 s1, p1) >> BN >> RELU [64k]
        self.presentation_layer = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.resnet_layer1 = ResnetBlock(in_channels=64, out_channels=128)

        self.norm_conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=256,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )

        self.resnet_layer2 = ResnetBlock(in_channels=256, out_channels=512)

        self.pooling = nn.MaxPool2d(kernel_size=(4, 4))

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        """forward pass of the resnet network"""
        op = self.presentation_layer(x)
        op = self.resnet_layer1(op)
        op = self.norm_conv1(op)
        op = self.resnet_layer2(op)
        op = self.pooling(op)
        op = self.classifier(op)
        return F.log_softmax(op, dim=-1)

In [ ]:
resnet_model = ResnetNetwork(in_channels=3, num_classes=10).to(device)
summary(resnet_model, input_size=(3, 32, 32))

In [ ]:
# updated training module---
def train_module(
    model: torch.nn.Module,
    device: torch.device,
    train_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    metric,
    train_losses: list,
    train_metrics: list,
):

    # setting model to train mode
    model.train()
    pbar = tqdm(train_dataloader)

    # batch metrics
    train_loss = 0
    train_metric = 0
    processed_batch = 0

    for idx, (data, label) in enumerate(pbar):
        # setting up device
        data = data.to(device)
        label = label.to(device)

        # forward pass output
        preds = model(data)

        # calc loss
        loss = criterion(preds, label)
        train_loss += loss.item()
        # print(f"training loss for batch {idx} is {loss}")

        # backpropagation
        optimizer.zero_grad()  # flush out  existing grads
        loss.backward()  # back prop of weights wrt loss
        optimizer.step()  # optimizer step -> minima

        # metric calc
        preds = torch.argmax(preds, dim=1)
        # print(f"preds:: {preds}")
        metric.update(preds, label)
        train_metric += metric.compute().detach().item()

        # updating batch count
        processed_batch += 1

        pbar.set_description(
            f"Avg Train Loss: {train_loss/processed_batch} Avg Train Metric: {train_metric/processed_batch}"
        )

    # It's typically called after the epoch completes
    metric.reset()
    # updating epoch metrics
    train_losses.append(train_loss / processed_batch)
    train_metrics.append(train_metric / processed_batch)

    return train_losses, train_metrics

In [ ]:
# updated testing modules---
def test_module(
    model: torch.nn.Module,
    device: torch.device,
    test_dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    metric,
    test_losses,
    test_metrics,
):
    # setting model to eval mode
    model.eval()
    pbar = tqdm(test_dataloader)

    # batch metrics
    test_loss = 0
    test_metric = 0
    processed_batch = 0

    with torch.inference_mode():
        for idx, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            # predictions
            preds = model(data)
            # print(preds.shape)
            # print(label.shape)

            # loss calc
            loss = criterion(preds, label)
            test_loss += loss.item()

            # metric calc
            preds = torch.argmax(preds, dim=1)
            metric.update(preds, label)
            test_metric += metric.compute().detach().item()

            # updating batch count
            processed_batch += 1

            pbar.set_description(
                f"Avg Test Loss: {test_loss/processed_batch} Avg Test Metric: {test_metric/processed_batch}"
            )

        # It's typically called after the epoch completes
        metric.reset()
        # updating epoch metrics
        test_losses.append(test_loss / processed_batch)
        test_metrics.append(test_metric / processed_batch)

    return test_losses, test_metrics

In [ ]:
# driver setup----
# optmizer
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)
# loss
criterion = nn.CrossEntropyLoss()
# metric
metric = MulticlassAccuracy(device=device, num_classes=10)
# Early stopping
early_stopping = EarlyStopping(
    patience=5, verbose=True, model_name="resnet_base_model.pt"
)

In [ ]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        resnet_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        resnet_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )


    early_stopping(
        test_losses[-1], resnet_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
plt.figure(figsize=(20, 11))
plot_loss_accuracy(
    train_loss=[train_losses],
    val_loss=[test_losses],
    train_acc=[train_metrics],
    val_acc=[test_metrics],
    labels=["base_resnet"],
    colors=["blue"],
    loss_legend_loc="upper left",
    acc_legend_loc="upper left",
    legend_font=7,
)

* The training accuracy has been around 91% where as we validation accuracy around 86%
* This could have been fixed using a couple of techniques like adding image augmentation , different weight init or adding regularization which all we will try later
* Next we will try to find a sweet value of LR using LR finder and then use cyclic LR

Redifining the model and drivers for Cyclic LR

In [ ]:
resnet_model = ResnetNetwork(in_channels=3, num_classes=10).to(device)
summary(resnet_model, input_size=(3, 32, 32))

In [ ]:
def model_drivers(
    model: nn.Module,
    learning_rate: float,
    num_classes: int,
    model_name: str,
    device: torch.device,
):
    """Initialize drivers for training"""
    # optmizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # loss
    criterion = nn.CrossEntropyLoss()
    # metric
    metric = MulticlassAccuracy(device=device, num_classes=num_classes)
    # Early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True, model_name=model_name)

    return model, optimizer, criterion, metric, early_stopping

In [ ]:
# optmizer
optimizer = optim.Adam(resnet_model.parameters(), lr=1e-5)
# loss
criterion = nn.CrossEntropyLoss()
# metric
metric = MulticlassAccuracy(device=device, num_classes=10)
# Early stopping
early_stopping = EarlyStopping(
    patience=5, verbose=True, model_name="resnet_base_model.pt"
)

In [ ]:
def suggested_lr(model, optimizer, criterion, device, end_lr, num_iter, train_loader):
    """Suggested learning rate"""
    lr_finder = LRFinder(model, optimizer, criterion, device=device)
    lr_finder.range_test(
        train_loader, end_lr=end_lr, num_iter=num_iter, step_mode="linear"
    )
    lr_finder.plot()
    lr_finder.reset()

In [ ]:
suggested_lr = 5.23E-03
print(suggested_lr)

In [ ]:
# adding a LR scheduler accordingly
scheduler = OneCycleLR(
    optimizer,
    max_lr=suggested_lr,
    epochs=epochs,
    steps_per_epoch=len(train_loader),
    anneal_strategy="linear",
    pct_start=0.10,
)

In [ ]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        resnet_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        resnet_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )
    # Lr scheduler added
    scheduler.step()
    print(f"Learning rate ahead: {scheduler.get_last_lr()}")

    early_stopping(
        test_losses[-1], resnet_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

lets use Step LR and ExponentialLR to check it we can improve

In [ ]:
resnet_model = ResnetNetwork(in_channels=3, num_classes=10).to(device)
summary(resnet_model, input_size=(3, 32, 32))

In [ ]:
# optmizer
optimizer = optim.Adam(resnet_model.parameters(), lr=suggested_lr)
# loss
criterion = nn.CrossEntropyLoss()
# metric
metric = MulticlassAccuracy(device=device, num_classes=10)
# Early stopping
early_stopping = EarlyStopping(
    patience=5, verbose=True, model_name="resnet_base_model.pt"
)
# Step LR
scheduler = StepLR(optimizer, step_size=5, gamma=0.95)  # decreae by 5 % after every 5 epochs

In [ ]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        resnet_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        resnet_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )
    # Lr scheduler added
    scheduler.step()
    print(f"Learning rate ahead: {scheduler.get_last_lr()}")

    early_stopping(
        test_losses[-1], resnet_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
#########################

In [ ]:
resnet_model = ResnetNetwork(in_channels=3, num_classes=10).to(device)
summary(resnet_model, input_size=(3, 32, 32))

In [ ]:
# optmizer
optimizer = optim.Adam(resnet_model.parameters(), lr=suggested_lr)
# loss
criterion = nn.CrossEntropyLoss()
# metric
metric = MulticlassAccuracy(device=device, num_classes=10)
# Early stopping
early_stopping = EarlyStopping(
    patience=5, verbose=True, model_name="resnet_base_model.pt"
)
# Step LR
scheduler = ExponentialLR( optimizer, gamma=0.95, verbose=True)  # exponential decay by 5 % after every epoch

In [ ]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        resnet_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        resnet_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )
    # Lr scheduler added
    scheduler.step()
    print(f"Learning rate ahead: {scheduler.get_last_lr()}")

    early_stopping(
        test_losses[-1], resnet_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
plt.figure(figsize=(20, 11))
plot_loss_accuracy(
    train_loss=[train_losses],
    val_loss=[test_losses],
    train_acc=[train_metrics],
    val_acc=[test_metrics],
    labels=["resnet_expLR"],
    colors=["blue"],
    loss_legend_loc="upper left",
    acc_legend_loc="upper left",
    legend_font=7,
)

In [4]:
def lr_scheduler(type, *args, **kwargs):
    if type == "StepLR":
        return StepLR(*args, **kwargs)
    elif type == "ExponentialLR":
        return ExponentialLR(*args, **kwargs)

    elif type == "CyclicLR":
        return OneCycleLR(*args, **kwargs)

Next we will complete the session 10 assigment in modular fashion